In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

def load_and_analyze_data(file_path):
    """Load the CSV and perform initial analysis"""
    df = pd.read_csv(file_path)
    print(f"Original dataset shape: {df.shape}")
    print(f"\nColumn names: {df.columns.tolist()}")
    print(f"\nData types:\n{df.dtypes}")
    print(f"\nNull values per column:\n{df.isnull().sum()}")
    return df

def clean_text_columns(df):
    """Clean text columns by removing extra whitespace and handling empty strings"""
    text_columns = ['title', 'director', 'cast', 'country', 'listed_in', 'description']

    for col in text_columns:
        if col in df.columns:
            # Replace empty strings and whitespace-only strings with NaN
            df[col] = df[col].astype(str).str.strip()
            df[col] = df[col].replace(['', 'nan', 'None'], np.nan)

    return df

def extract_duration_minutes(duration_str):
    """Extract duration in minutes from duration string"""
    if pd.isna(duration_str):
        return np.nan

    duration_str = str(duration_str).strip()

    # For movies (e.g., "90 min")
    if 'min' in duration_str:
        match = re.search(r'(\d+)\s*min', duration_str)
        return int(match.group(1)) if match else np.nan

    # For TV shows (e.g., "2 Seasons")
    elif 'Season' in duration_str:
        match = re.search(r'(\d+)\s*Season', duration_str)
        # Convert seasons to approximate minutes (assuming 10 episodes per season, 45 min each)
        return int(match.group(1)) * 450 if match else np.nan

    return np.nan

def detect_outliers_iqr(data, column):
    """Detect outliers using IQR method"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

def preprocess_netflix_data(file_path, remove_outliers=True, outlier_method='iqr'):
    """
    Main preprocessing function

    Parameters:
    - file_path: path to the CSV file
    - remove_outliers: whether to remove outliers (default: True)
    - outlier_method: method for outlier detection ('iqr' or 'zscore')
    """

    # Load and analyze data
    df = load_and_analyze_data(file_path)
    original_shape = df.shape

    print("\n" + "="*50)
    print("PREPROCESSING STEPS")
    print("="*50)

    # Step 1: Clean text columns
    print("\n1. Cleaning text columns...")
    df = clean_text_columns(df)

    # Step 2: Handle date columns
    print("2. Processing date columns...")
    if 'date_added' in df.columns:
        # Convert date_added to datetime
        df['date_added'] = pd.to_datetime(df['date_added'], format='%d-%b-%y', errors='coerce')

    # Step 3: Process duration column
    print("3. Processing duration column...")
    if 'duration' in df.columns:
        df['duration_minutes'] = df['duration'].apply(extract_duration_minutes)

    # Step 4: Handle release_year outliers
    print("4. Checking for outliers in release_year...")
    current_year = datetime.now().year
    if 'release_year' in df.columns:
        # Remove unrealistic release years (before 1900 or after current year + 5)
        year_outliers = df[(df['release_year'] < 1900) | (df['release_year'] > current_year + 5)]
        print(f"   Found {len(year_outliers)} unrealistic release years")
        if len(year_outliers) > 0:
            print(f"   Outlier years: {year_outliers['release_year'].unique()}")
            df = df[(df['release_year'] >= 1900) & (df['release_year'] <= current_year + 5)]

    # Step 5: Handle duration outliers
    if remove_outliers and 'duration_minutes' in df.columns:
        print("5. Detecting duration outliers...")
        duration_data = df.dropna(subset=['duration_minutes'])

        if len(duration_data) > 0:
            outliers, lower_bound, upper_bound = detect_outliers_iqr(duration_data, 'duration_minutes')
            print(f"   Duration bounds: {lower_bound:.1f} - {upper_bound:.1f} minutes")
            print(f"   Found {len(outliers)} duration outliers")

            if len(outliers) > 0:
                print(f"   Outlier durations: {sorted(outliers['duration_minutes'].unique())}")
                # Remove duration outliers
                df = df[(df['duration_minutes'].isna()) |
                       ((df['duration_minutes'] >= lower_bound) & (df['duration_minutes'] <= upper_bound))]

    # Step 6: Remove rows with too many null values
    print("6. Removing rows with excessive null values...")
    # Calculate null percentage per row
    null_percentage = df.isnull().sum(axis=1) / len(df.columns)
    rows_to_remove = df[null_percentage > 0.5]  # Remove rows with >50% null values
    print(f"   Removing {len(rows_to_remove)} rows with >50% null values")
    df = df[null_percentage <= 0.5]

    # Step 7: Handle specific null values based on content type
    print("7. Handling nulls based on content type...")

    # For movies, director is more important; for TV shows, cast might be more flexible
    movies = df[df['type'] == 'Movie']
    tv_shows = df[df['type'] == 'TV Show']

    print(f"   Movies: {len(movies)}, TV Shows: {len(tv_shows)}")

    # Remove movies without title, director, or description
    if 'director' in df.columns:
        movies_to_keep = movies.dropna(subset=['title', 'description'])
        # For movies, we're more lenient with director nulls since some documentaries might not have traditional directors

    # Remove TV shows without title or description
    tv_shows_to_keep = tv_shows.dropna(subset=['title', 'description'])

    # Combine cleaned data
    df_cleaned = pd.concat([movies_to_keep, tv_shows_to_keep], ignore_index=True)

    # Final summary
    print("\n" + "="*50)
    print("PREPROCESSING SUMMARY")
    print("="*50)
    print(f"Original shape: {original_shape}")
    print(f"Final shape: {df_cleaned.shape}")
    print(f"Rows removed: {original_shape[0] - df_cleaned.shape[0]}")
    print(f"Removal percentage: {((original_shape[0] - df_cleaned.shape[0]) / original_shape[0] * 100):.1f}%")

    print(f"\nRemaining null values per column:")
    remaining_nulls = df_cleaned.isnull().sum()
    for col, nulls in remaining_nulls.items():
        if nulls > 0:
            print(f"   {col}: {nulls} ({nulls/len(df_cleaned)*100:.1f}%)")

    return df_cleaned

def save_cleaned_data(df, output_path):
    """Save the cleaned dataset"""
    df.to_csv(output_path, index=False)
    print(f"\nCleaned data saved to: {output_path}")

if __name__ == "__main__":
    input_file = 'netflix_titles.csv'
    output_file = 'netflix_cleaned.csv'

    try:
        cleaned_df = preprocess_netflix_data(input_file, remove_outliers=True)

        save_cleaned_data(cleaned_df, output_file)

        print("\nSample of cleaned data:")
        print(cleaned_df.head())

        print("\nFinal data types:")
        print(cleaned_df.dtypes)

    except FileNotFoundError:
        print(f"Error: Could not find file '{input_file}'. Please check the file path.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Original dataset shape: (8807, 12)

Column names: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']

Data types:
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

Null values per column:
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

PREPROCESSING STEPS

1. Cleaning text columns...
2. Processing date columns...
3. Processing duration column...
4. Checking for outliers in release_year...
   Found 0 unrealistic release years
5. Detecting duration o